<a href="https://colab.research.google.com/github/jpbeaud/language/blob/main/essai1_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install keras-preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00


Création & préparation du modèle

In [8]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Définition des paramètres
num_encoder_tokens = 100
num_decoder_tokens = 100
latent_dim = 256

# Encodeur
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]
print(encoder_inputs)
# Décodeur
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Modèle Seq2Seq
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


<KerasTensor shape=(None, None, 100), dtype=float32, sparse=False, name=keras_tensor_9>


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, None, 100)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, None, 100)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 256), (None,   │        365,568 │ input_layer_2[0][0]    │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 256),    │        365,568 │ input_layer_3[0][0],   │
│                           │ (None, 256), (None,    │                │ lstm_2[0][1],          │
│                           │ 256)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, None, 100)      │         25,700 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 756,836 (2.89 MB)

 Trainable params: 756,836 (2.89 MB)

 Non-trainable params: 0 (0.00 B)

Préparation des données

In [2]:

from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import numpy as np



# Exemples de données (à remplacer par tes propres données)
input_texts = ["bonjour", "salut"]
target_texts = ["hello", "hi"]

# Tokenisation des textes
input_tokenizer = Tokenizer(num_words=num_encoder_tokens)
print("num_encorder_tokens = ", num_encoder_tokens)
print("num_decoder_tokens = ", num_decoder_tokens)
input_tokenizer.fit_on_texts(input_texts)
encoder_input_data = input_tokenizer.texts_to_sequences(input_texts)
encoder_input_data = pad_sequences(encoder_input_data, padding='post')

target_tokenizer = Tokenizer(num_words=num_decoder_tokens)
target_tokenizer.fit_on_texts(target_texts)
decoder_input_data = target_tokenizer.texts_to_sequences(target_texts)
decoder_input_data = pad_sequences(decoder_input_data, padding='post')


num_encorder_tokens =  100
num_decoder_tokens =  100


Entrainement du modèle avec les données préparées

In [14]:
# Convertir les données cibles en une représentation à une dimension
decoder_target_data = np.zeros((len(target_texts), max([len(t) for t in decoder_input_data]), num_decoder_tokens))
for i, seqs in enumerate(decoder_input_data):
    #print(i, seqs)
    for t, token in enumerate(seqs):
        #print(t, token)
        if token.all() > 0:
            decoder_target_data[i, t, token] = 1.0

# Entraîner le modèle
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=100,
          validation_split=0.2)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.0000e+00 